# Phase 4, Lesson 1: GPT Architecture[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/suraaj3poudel/Learn-To-Make-GPT-Model/blob/main/phase4_build_your_gpt/01_gpt_architecture.ipynb)Build your own GPT from scratch! 🤖## What You'll Learn1. GPT architecture (decoder-only transformer)2. Modern techniques (dropout, weight tying, etc.)3. Using PyTorch for real implementation4. Model configuration and scalingThe final boss! Let's do this!

In [ ]:
# Setup with PyTorchimport torchimport torch.nn as nnimport torch.nn.functional as Fimport numpy as npimport matplotlib.pyplot as pltprint(f'✅ PyTorch version: {torch.__version__}')print(f'✅ Device: {"cuda" if torch.cuda.is_available() else "cpu"}')

## 1. GPT vs. Full Transformer**Key Differences**:- GPT = **Decoder-only** (no encoder)- Uses **causal/masked** attention (can't see future)- Trained on **autoregressive** language modeling**Architecture**:```Input tokens    ↓Token + Position Embeddings    ↓Transformer Block 1    ↓Transformer Block 2    ↓... (many blocks)    ↓Transformer Block N    ↓Layer Norm    ↓Linear (to vocabulary)    ↓Output probabilities```

## 2. ConfigurationDefine model hyperparameters

In [ ]:
class GPTConfig:    """Configuration for GPT model"""    def __init__(        self,        vocab_size=50257,      # GPT-2 vocabulary size        n_positions=1024,       # Maximum sequence length        n_embd=768,            # Embedding dimension        n_layer=12,            # Number of transformer blocks        n_head=12,             # Number of attention heads        dropout=0.1,           # Dropout probability    ):        self.vocab_size = vocab_size        self.n_positions = n_positions        self.n_embd = n_embd        self.n_layer = n_layer        self.n_head = n_head        self.dropout = dropout# Example configurationsconfig_small = GPTConfig(    vocab_size=10000,    n_positions=256,    n_embd=128,    n_layer=4,    n_head=4,)config_medium = GPTConfig(    vocab_size=50257,    n_positions=1024,    n_embd=768,    n_layer=12,    n_head=12,)print("Small GPT config:")print(f"  Params: ~{(config_small.n_layer * config_small.n_embd ** 2 * 12) / 1e6:.1f}M")print(f"  Embedding dim: {config_small.n_embd}")print(f"  Layers: {config_small.n_layer}")

## 3. Multi-Head Attention (PyTorch)Professional implementation with PyTorch

In [ ]:
class MultiHeadAttention(nn.Module):    def __init__(self, config):        super().__init__()        assert config.n_embd % config.n_head == 0                # Key, query, value projections for all heads (in one matrix)        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)                # Output projection        self.c_proj = nn.Linear(config.n_embd, config.n_embd)                # Dropout        self.attn_dropout = nn.Dropout(config.dropout)        self.resid_dropout = nn.Dropout(config.dropout)                self.n_head = config.n_head        self.n_embd = config.n_embd                # Causal mask (lower triangular)        self.register_buffer(            "bias",            torch.tril(torch.ones(config.n_positions, config.n_positions))                  .view(1, 1, config.n_positions, config.n_positions)        )        def forward(self, x):        B, T, C = x.size()  # batch, sequence length, embedding dim                # Calculate Q, K, V for all heads in batch        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)                # Split into multiple heads        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)                # Attention: (Q @ K^T) / sqrt(d_k)        att = (q @ k.transpose(-2, -1)) * (1.0 / torch.sqrt(torch.tensor(k.size(-1))))        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))        att = F.softmax(att, dim=-1)        att = self.attn_dropout(att)                # Apply attention to values        y = att @ v  # (B, nh, T, hs)        y = y.transpose(1, 2).contiguous().view(B, T, C)  # Re-assemble all head outputs                # Output projection        y = self.resid_dropout(self.c_proj(y))        return y# Testconfig = config_smallmha = MultiHeadAttention(config)x = torch.randn(2, 10, config.n_embd)  # Batch of 2, sequence of 10output = mha(x)print(f"Input shape: {x.shape}")print(f"Output shape: {output.shape}")print("✅ Multi-head attention working!")

## 4. Feed-Forward NetworkMLP with GELU activation (used in GPT)

In [ ]:
class FeedForward(nn.Module):    def __init__(self, config):        super().__init__()        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)        self.dropout = nn.Dropout(config.dropout)        self.gelu = nn.GELU()        def forward(self, x):        x = self.c_fc(x)        x = self.gelu(x)        x = self.c_proj(x)        x = self.dropout(x)        return x# Testffn = FeedForward(config)x = torch.randn(2, 10, config.n_embd)output = ffn(x)print(f"Input shape: {x.shape}")print(f"Output shape: {output.shape}")

## 5. Transformer BlockCombine attention and feed-forward with residuals and layer norm

In [ ]:
class TransformerBlock(nn.Module):    def __init__(self, config):        super().__init__()        self.ln_1 = nn.LayerNorm(config.n_embd)        self.attn = MultiHeadAttention(config)        self.ln_2 = nn.LayerNorm(config.n_embd)        self.mlp = FeedForward(config)        def forward(self, x):        # Attention with residual        x = x + self.attn(self.ln_1(x))                # Feed-forward with residual        x = x + self.mlp(self.ln_2(x))                return x# Testblock = TransformerBlock(config)x = torch.randn(2, 10, config.n_embd)output = block(x)print(f"Input shape: {x.shape}")print(f"Output shape: {output.shape}")print("✅ Transformer block working!")

## 6. Complete GPT ModelPut it all together!

In [ ]:
class GPT(nn.Module):    def __init__(self, config):        super().__init__()        self.config = config                # Token + position embeddings        self.transformer = nn.ModuleDict(dict(            wte = nn.Embedding(config.vocab_size, config.n_embd),  # Token embeddings            wpe = nn.Embedding(config.n_positions, config.n_embd),  # Position embeddings            drop = nn.Dropout(config.dropout),            h = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layer)]),            ln_f = nn.LayerNorm(config.n_embd),        ))                # Language model head        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)                # Weight tying (share weights between token embedding and output)        self.transformer.wte.weight = self.lm_head.weight                # Initialize weights        self.apply(self._init_weights)        def _init_weights(self, module):        if isinstance(module, nn.Linear):            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)            if module.bias is not None:                torch.nn.init.zeros_(module.bias)        elif isinstance(module, nn.Embedding):            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)        def forward(self, idx, targets=None):        device = idx.device        b, t = idx.size()        assert t <= self.config.n_positions, f"Sequence length {t} exceeds maximum {self.config.n_positions}"                # Position indices        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0)  # (1, t)                # Forward pass        tok_emb = self.transformer.wte(idx)  # Token embeddings: (b, t, n_embd)        pos_emb = self.transformer.wpe(pos)  # Position embeddings: (1, t, n_embd)        x = self.transformer.drop(tok_emb + pos_emb)                # Transformer blocks        for block in self.transformer.h:            x = block(x)                x = self.transformer.ln_f(x)                # Language model head        logits = self.lm_head(x)  # (b, t, vocab_size)                # Calculate loss if targets provided        loss = None        if targets is not None:            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))                return logits, loss        def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):        """        Generate text autoregressively                Args:            idx: (b, t) array of indices            max_new_tokens: How many tokens to generate            temperature: Sampling temperature            top_k: If set, only sample from top k tokens        """        for _ in range(max_new_tokens):            # Crop to max length            idx_cond = idx if idx.size(1) <= self.config.n_positions else idx[:, -self.config.n_positions:]                        # Forward pass            logits, _ = self(idx_cond)                        # Get logits for last position            logits = logits[:, -1, :] / temperature                        # Optional: crop to top k            if top_k is not None:                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))                logits[logits < v[:, [-1]]] = -float('Inf')                        # Sample            probs = F.softmax(logits, dim=-1)            idx_next = torch.multinomial(probs, num_samples=1)                        # Append to sequence            idx = torch.cat((idx, idx_next), dim=1)                return idx# Create modelconfig = config_smallmodel = GPT(config)# Count parametersn_params = sum(p.numel() for p in model.parameters())print(f"GPT model created!")print(f"Parameters: {n_params:,}")print(f"Model size: ~{n_params/1e6:.2f}M parameters")# Test forward passx = torch.randint(0, config.vocab_size, (2, 10))logits, loss = model(x, targets=x)print(f"\nInput shape: {x.shape}")print(f"Output logits shape: {logits.shape}")print(f"\n✅ Complete GPT model working!")

## 7. Model SizesCompare different GPT scales:

In [ ]:
# Different model configurationsconfigs = {    "Tiny": GPTConfig(vocab_size=10000, n_embd=64, n_layer=2, n_head=2, n_positions=128),    "Small": GPTConfig(vocab_size=10000, n_embd=128, n_layer=4, n_head=4, n_positions=256),    "Medium": GPTConfig(vocab_size=10000, n_embd=256, n_layer=6, n_head=8, n_positions=512),    "GPT-2 Small": GPTConfig(vocab_size=50257, n_embd=768, n_layer=12, n_head=12, n_positions=1024),}print("Model Size Comparison:\n")for name, cfg in configs.items():    model = GPT(cfg)    n_params = sum(p.numel() for p in model.parameters())    print(f"{name:15} {n_params/1e6:8.2f}M parameters")print("\nGPT-3 has 175 BILLION parameters! 🤯")

## Summary### What We Built:1. **GPT Configuration** - Flexible model sizing2. **Multi-Head Attention** - Professional PyTorch implementation3. **Transformer Blocks** - With residuals and layer norm4. **Complete GPT Model** - Ready for training!5. **Generation Method** - Autoregressive sampling### Key Insights:- GPT = Stack of transformer blocks- Weight tying saves parameters- Causal masking prevents cheating- Scale matters (but small models can still work!)### Next Steps:👉 **Lesson 2**: Train this GPT on real data!You built GPT from scratch! 🎉